In [1]:
%matplotlib inline
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
import json
import csv
import pandas as pd
import sklearn.feature_extraction.text as sk_text
import io
import requests
import numpy as np
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics

#------------------PROFESSOR'S CODE, HIDDEN GEM--------------------------
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

#fix headers for census data
def fix_table(df,dropig=True):
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    df = df.astype({"Id2" : int})
    if dropig:
        df = df.drop(['Id','Geography'],axis=1)
    return df


In [4]:
dp1  = pd.read_csv('census_data/DEC_00_110H_DP1_with_ann.csv', encoding="utf-8")
dp2  = pd.read_csv('census_data/DEC_00_110S_DP2_with_ann.csv', encoding="utf-8")
dp3  = pd.read_csv('census_data/DEC_00_110S_DP3_with_ann.csv', encoding="utf-8")
dp4  = pd.read_csv('census_data/DEC_00_110S_DP4_with_ann.csv', encoding="utf-8")
h002 = pd.read_csv('census_data/DEC_00_110H_H002_with_ann.csv', encoding="utf-8")
res  = pd.read_csv('census_data/election_results.csv', encoding="utf-8")

#fix headers
dp1  = fix_table(dp1,False)
dp2  = fix_table(dp2)
dp3  = fix_table(dp3)
dp4  = fix_table(dp4)
h002 = fix_table(h002)
res  = fix_table(res)

df = dp1.copy()
df = df.merge(dp2,on="Id2")
df = df.merge(dp3,on="Id2")
df = df.merge(dp4,on="Id2")
df = df.merge(h002,on="Id2")
df = df.merge(res,on="Id2")

df

,Id,Id2,Geography,Number; Total population,Percent; Total population,Number; Total population - SEX AND AGE - Male,Percent; Total population - SEX AND AGE - Male,Number; Total population - SEX AND AGE - Female,Percent; Total population - SEX AND AGE - Female,Number; Total population - SEX AND AGE - Under 5 years,...,Percent; Specified renter-occupied units - GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN 1999 - 35 percent or more,Number; Specified renter-occupied units - GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN 1999 - Not computed,Percent; Specified renter-occupied units - GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN 1999 - Not computed,Total:,Urban:,Urban: - Inside urbanized areas,Urban: - Inside urban clusters,Rural,Not defined for this file,winner
0,5001000US0101,101,"Congressional District 1 (110th Congress), Ala...",635300,100.0,306620,48.3,328680,51.7,44487,...,29.5,8395,13.1,281691,175729,133997,41732,105962,0,R
1,5001000US0102,102,"Congressional District 2 (110th Congress), Ala...",635300,100.0,308305,48.5,326995,51.5,42135,...,25.7,10306,15.0,281290,136673,82178,54495,144617,0,R
2,5001000US0103,103,"Congressional District 3 (110th Congress), Ala...",635300,100.0,305852,48.1,329448,51.9,40841,...,31.3,10631,15.0,286010,150521,113943,36578,135489,0,R
3,5001000US0104,104,"Congressional District 4 (110th Congress), Ala...",635300,100.0,310279,48.8,325021,51.2,40985,...,20.9,10336,19.7,282766,76421,31756,44665,206345,0,R
4,5001000US0105,105,"Congressional District 5 (110th Congress), Ala...",635300,100.0,309694,48.7,325606,51.3,41451,...,25.6,7688,11.3,278684,169089,147823,21266,109595,0,D
5,5001000US0106,106,"Congressional District 6 (110th Congress), Ala...",635300,100.0,310240,48.8,325060,51.2,41352,...,22.9,6218,11.5,271212,169584,154689,14895,101628,0,R
6,5001000US0107,107,"Congressional District 7 (110th Congress), Ala...",635300,100.0,295514,46.5,339786,53.5,44741,...,31.8,11932,13.1,282058,202666,186106,16560,79392,0,D
7,5001000US0200,200,Congressional District (at Large) (110th Congr...,626932,100.0,324112,51.7,302820,48.3,47591,...,25.4,11534,14.0,260978,157834,108417,49417,103144,0,R
8,5001000US0401,401,"Congressional District 1 (110th Congress), Ari...",641329,100.0,325515,50.8,315814,49.2,44745,...,28.2,8408,13.4,293918,149292,51821,97471,144626,0,R
9,5001000US0402,402,"Congressional District 2 (110th Congress), Ari...",641329,100.0,313683,48.9,327646,51.1,40887,...,30.8,4956,9.6,287665,253368,188931,64437,34297,0,R
